In [4]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=1,suppress=True)

# Sujet 02 : Régime Alimentaire

On utilisera les données contenues dans le fichier `Aliments.csv` dans la suite.

1. Les besoins  journaliers de Marie sont les suivants :
- Kcal : 2000kcal (F)
- Protéines : 75g (F)
- Glucides : 225g (F)
- Lipides : 90g (F)
- Fer : 9mg (F) 
- Calcium : 0.8g (F)
- Fibre : 45g (F)

**Comment doit-elle satisfaire ses besoins si elle veut minimiser son budget?**

2. Comment procéder si elle ne veut pas dépasser de plus de 10% les apports journaliers?

Reformumation du problème et mise en place des, étapes à suivre.
On considère que les régimes ne doivent pas apporter moins que ces quantités.

1) Proposer le régime journalier le moins cher pour marie et pour les femmes en général.

2) Même question en considérant que les différents critéres ne doivent pas dépasser 1.1 fois (10%) la dose recommandé.

3) Dans le but d 'un régime, on souhaite minimiser son budget, mais aussi minimiser les calories sans s'opposer aux autres contraintes. Le faire sans contraintes de budget (H et F) puis avec différents niveaux de contraintes journalières. 

4) Faire une étude détaillé  du menu proposé.

On commence par charger le fichier de donnée dans un data frame (avec panda)

In [5]:
Al=pd.read_csv('./Aliments.csv',sep=';',index_col=0)

In [6]:
Al.head()

,Protéines (en g),Lipides (en g),Glucides (en g),Energie (kcal),Fer (mg),Calcium (mg),Fibres (en g),Prix €/100g
Produit (100g),,,,,,,,
Lait – Emmental,26.93,27.80,5.38,380,0.20,791.0,0.0,1.30
Lait – crème fraîche,20.90,35.00,2.84,328,0.03,66.3,0.0,0.84
Lait – Petit-suisse,9.40,9.45,3.30,136,0.20,112.0,0.0,0.44
Lait – Lait entier (vache),3.15,3.25,4.80,61,0.30,113.0,0.0,0.28
Lait – Lait entier (chèvre),3.56,4.14,4.45,69,0.05,134.0,0.0,0.70


# Question 1

a) Détermination des matrices de travail.

In [7]:
A=np.array(Al).T
coeff=A[:-1] ## matrice des coefficients
z_ori=A[-1]### coefficients de la fonction de coût (prix)
betaF=np.array([75,90,225,2000,9,800,45]) ## second membre pour les femmes
# 2000kcal, 75g prot, 225g glu, 90g lip, 9mg fer, 800mg calc, 45g fibre

## Formalisation mathématique du problème

N = 41 Nombres d'aliments

P = 8 Nombre de nutriments

$\beta i$  $\forall i \in [1;P]$ est le besoin journalier en nutriments

$C_{ij}$ $\forall i \in [1;N]$ et $\forall j \in [1,P]$ est l'apport du i-ème aliment par rapport au j-ème nutriment : soit 100g du i-ème aliment apporte $C_{ij}$ grammes du j-ème nutriment.

$Pi$ $\forall i \in [1;N]$ est le prix au 100g du i-ème aliment.

$qi$ $\forall$ i $\in [1;N]$ est la quantité achetée/consommée du i-ème aliment.

$\sum_{i=1}^{N} C_{ij} q_i \geq$ $\beta_j$   $\forall j \in [1;P]$ avec $q_i \geq 0$   $\forall i \in [1;N]$

$z$ = $\min \sum_{i=1}^{N} p_i q_i$ est le coût minimum du repas.

**Traduction en linprog**

z_ori = z ce sont les coéffiscients d'une fonction linéaire à minimiser, ici c'est le vecteur de la 8-ème colonne contennant les prix ($Pi$).

A_ub = -coeff est une matrice (tableau-2D) contenant les coéficients de la contrainte d'inégalité, contraints en $x$ avec A_ub $\times$ $x$ $\leq$ b_ub, c'est pourquoi ici on a mis un moins devant coeff car nous on veut que les besoins journaliers soient au minimum satisfait si ce n'est plus. Précédement appellé $C_{ij}$ $\times$ $qi$

b_ub = -betaF est le vecteur des besoins journaliers en nutriments. Préccédement noté $\beta i$. Là encore on a mis un moins devant coeff car nous on veut que les besoins journaliers soient au minimum satisfait si ce n'est plus.

bounds = les contraintes de quantités que l'on pourra imposer ou non à un ou plusieurs aliments.

b) On utilise scipy.optimize

In [8]:
import scipy.optimize as so

In [9]:
RESULTF=so.linprog(z_ori, A_ub=-coeff, b_ub=-betaF,method='simplex') 
RESULTF
so.linprog

C:\Users\karl\AppData\Local\Temp\ipykernel_16360\2621746166.py:1: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  RESULTF=so.linprog(z_ori, A_ub=-coeff, b_ub=-betaF,method='simplex')


<function scipy.optimize._linprog.linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='highs', callback=None, options=None, x0=None, integrality=None)>

In [10]:
RESULTF.fun ##cout minimal

2.052827380785162

In [11]:
A=RESULTF.x ##valeurs réalisant l'optimum
A

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.3,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. , 13.5,  0. ,  0. ])

b) Calculons maintenant le data frame calculant les apports de ce repas.

In [12]:
u,=A.nonzero()
REST=Al.iloc[u,:]
display(REST)
QTE=pd.DataFrame({'Quantités en g' : 100*A[A.nonzero()]}).set_index(Al.iloc[u,:].index)
display(QTE)

,Protéines (en g),Lipides (en g),Glucides (en g),Energie (kcal),Fer (mg),Calcium (mg),Fibres (en g),Prix €/100g
Produit (100g),,,,,,,,
Lait – Beurre,0.85,81.11,0.06,717,0.02,24.0,0.0,1.18
Base – Haricots blancs,5.54,5.15,21.63,155,1.99,61.0,5.5,0.13


,Quantités en g
Produit (100g),
Lait – Beurre,25.248785
Base – Haricots blancs,1349.916703


In [13]:

BILAN=pd.DataFrame((np.array(REST).T*A[A.nonzero()]).T).set_index(QTE.index)
REPAS=pd.concat([QTE,BILAN],axis=1)
REPAS=REPAS.append([REPAS.sum()])
Namescol=['Quantité (en g)']+list(Al.columns)
Namescol[-1]='Prix (en €)'
REPAS.columns=Namescol
Namesrow=list(QTE.index)
Namesrow.append('Apports du repas')
REPAS.index=Namesrow
display(REPAS)

C:\Users\karl\AppData\Local\Temp\ipykernel_16360\921585303.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  REPAS=REPAS.append([REPAS.sum()])


,Quantité (en g),Protéines (en g),Lipides (en g),Glucides (en g),Energie (kcal),Fer (mg),Calcium (mg),Fibres (en g),Prix (en €)
Lait – Beurre,25.248785,0.214615,20.47929,0.015149,181.033791,0.005050,6.059708,0.000000,0.297936
Base – Haricots blancs,1349.916703,74.785385,69.52071,291.986983,2092.370889,26.863342,823.449189,74.245419,1.754892
Apports du repas,1375.165488,75.000000,90.00000,292.002132,2273.404680,26.868392,829.508897,74.245419,2.052827


On en fait une fonction

In [14]:
def apports(RESULT):
    A=RESULT.x
    u,=A.nonzero()
    REST=Al.iloc[u,:]
    QTE=pd.DataFrame({'Quantités en g' : 100*A[A.nonzero()]}).set_index(Al.iloc[u,:].index)
    BILAN=pd.DataFrame((np.array(REST).T*A[A.nonzero()]).T).set_index(QTE.index)
    REPAS=pd.concat([QTE,BILAN],axis=1)
    REPAS=REPAS.append([REPAS.sum()])
    Namescol=['Quantité (en g)']+list(Al.columns)
    Namescol[-1]='Prix (en €)'
    REPAS.columns=Namescol
    Namesrow=list(QTE.index)
    Namesrow.append('Apports du repas')
    REPAS.index=Namesrow

    return REPAS

In [15]:
type(REPAS)

pandas.core.frame.DataFrame

In [16]:
REPAS=apports(RESULTF)
display(REPAS.iloc[-1,0])
REPAS

C:\Users\karl\AppData\Local\Temp\ipykernel_16360\2694851566.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  REPAS=REPAS.append([REPAS.sum()])


1375.165487978005

,Quantité (en g),Protéines (en g),Lipides (en g),Glucides (en g),Energie (kcal),Fer (mg),Calcium (mg),Fibres (en g),Prix (en €)
Lait – Beurre,25.248785,0.214615,20.47929,0.015149,181.033791,0.005050,6.059708,0.000000,0.297936
Base – Haricots blancs,1349.916703,74.785385,69.52071,291.986983,2092.370889,26.863342,823.449189,74.245419,1.754892
Apports du repas,1375.165488,75.000000,90.00000,292.002132,2273.404680,26.868392,829.508897,74.245419,2.052827


c) Présentation des résultats

In [17]:
##Présentation naive##
u,=A.nonzero()
for k in u :
    print(Al.index[k])
    print('{:0.2f} g'.format(A[k]*100))

Lait – Beurre
25.25 g
Base – Haricots blancs
1349.92 g


In [18]:
##Cette fonction permet d'enlever le type de chaque aliment
def rename_aliment(old_name):
    new_name=old_name.replace("Lait – ", "")
    new_name=new_name.replace("Viande – ", "")
    new_name=new_name.replace("Poisson – ", "")
    new_name=new_name.replace("Légume – ", "")
    new_name=new_name.replace("Base – ", "")
    return new_name

In [19]:
Phrase='Un repas est constitué de '
for s in range(len(u)-1):
    if s>0 : Phrase+=','
    gr=A[A.nonzero()][s]*100
    old_name=Al.iloc[u].index[s]
    name = rename_aliment(old_name)
    Phrase+=' de {:0.2f} g de {}'.format(gr,name)
s=len(u)-1    
gr=A[A.nonzero()][s]*100
old_name=Al.iloc[u].index[s]
name = rename_aliment(old_name)
Phrase+=' et de {:0.2f} g de {}. '.format(gr,name)
Phrase+=' Il coûte un total de {:0.2f} euros.'.format(RESULTF.fun)
print(Phrase)

Un repas est constitué de  de 25.25 g de Beurre et de 1349.92 g de Haricots blancs.  Il coûte un total de 2.05 euros.


Mettons cela dans une fonction.

In [20]:
def repas(RESULT):
    A=RESULT.x
    u,=A.nonzero()
    Qt=A[u]
    Phrase='Un repas est constitué de '
    for s in range(len(u)-1):
        if s>0 : Phrase+=','
        gr=Qt[s]*100
        old_name=Al.iloc[u].index[s]
        name = rename_aliment(old_name)
        Phrase+=' de {:0.2f} g de {}'.format(gr,name)
    s=len(u)-1    
    gr=Qt[s]*100
    old_name=Al.iloc[u].index[s]
    name = rename_aliment(old_name)
    Phrase+=' et de {:0.2f} g de {}. '.format(gr,name)
    REPAS=apports(RESULT)
    Phrase+=' Il coûte un total de {:0.2f} euros et comprend {:0.2f} calories.'.format(RESULT.fun,REPAS['Energie (kcal)'][-1])
    return Phrase

In [21]:
# RESULTF=so.linprog(z_ori, A_ub=-coeff, b_ub=-betaF,method='simplex') 
print('Pour les femmes.')
repas(RESULTF)

Pour les femmes.


C:\Users\karl\AppData\Local\Temp\ipykernel_16360\2694851566.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  REPAS=REPAS.append([REPAS.sum()])


'Un repas est constitué de  de 25.25 g de Beurre et de 1349.92 g de Haricots blancs.  Il coûte un total de 2.05 euros et comprend 2273.40 calories.'

In [45]:
A=np.array(Al).T
coeff=A[:-1] ## matrice des coefficients
z_ori=A[-1]### coefficients de la fonction de coût (prix)
betaF=np.array([75,90,225,2000,9,800,45]) ## second membre pour les femmes
# 2000kcal, 75g prot, 225g glu, 90g lip, 9mg fer, 800mg calc, 45g fibre
bdns = [(0,None) for k in range(41)]
bdns[38] = (0,5)

In [46]:
RESULTF2=so.linprog(z_ori, A_ub=-coeff, b_ub=-betaF,method='simplex', bounds = bdns) 
repas(RESULTF2)

C:\Users\karl\AppData\Local\Temp\ipykernel_16360\2198535615.py:1: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  RESULTF2=so.linprog(z_ori, A_ub=-coeff, b_ub=-betaF,method='simplex', bounds = bdns)
C:\Users\karl\AppData\Local\Temp\ipykernel_16360\2694851566.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  REPAS=REPAS.append([REPAS.sum()])


'Un repas est constitué de  et de 0.10 g de Gruyère.  Il coûte un total de 0.00 euros et comprend 0.41 calories.'

# Question 2

In [24]:
RESULTFb=so.linprog(z_ori, A_ub=np.concatenate((-coeff,coeff),axis=0), b_ub=np.concatenate((-betaF,1.1*betaF),axis=0),method='simplex') 
print('Pour les femmes avec des contraintes supplémentaires.')
repas(RESULTFb)

Pour les femmes avec des contraintes supplémentaires.


C:\Users\karl\AppData\Local\Temp\ipykernel_16360\2910164087.py:1: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  RESULTFb=so.linprog(z_ori, A_ub=np.concatenate((-coeff,coeff),axis=0), b_ub=np.concatenate((-betaF,1.1*betaF),axis=0),method='simplex')
C:\Users\karl\AppData\Local\Temp\ipykernel_16360\2694851566.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  REPAS=REPAS.append([REPAS.sum()])


'Un repas est constitué de  de 119.21 g de Beurre et de 679.81 g de Courge.  Il coûte un total de 3.58 euros et comprend 1235.41 calories.'

In [25]:
np.concatenate((-betaF,1.5*betaF),axis=0)

array([   -5. ,   -90. ,  -225. , -2000. ,    -9. ,  -800. ,   -45. ,
           7.5,   135. ,   337.5,  3000. ,    13.5,  1200. ,    67.5])

Analyse additionelle en s'approchant au mieux du minimum.


In [26]:
for k in [30,20,10,5,4,3,2,1.5,1.4,1.3,1.2,1.1,1.09,1.08,1.07]:
    RESULTFb=so.linprog(z_ori, A_ub=np.concatenate((-coeff,coeff),axis=0), b_ub=np.concatenate((-betaF,k*betaF),axis=0),method='simplex') 
    if RESULTFb.success==True:
        print('\nPour les femmes avec des contraintes supplémentaires entre le minimum et {} fois le minimum.'.format(k))
        repas(RESULTFb)
    else : print('\nRepas impossible pour {} fois le minimum'.format(k))        

C:\Users\karl\AppData\Local\Temp\ipykernel_16360\980770844.py:2: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  RESULTFb=so.linprog(z_ori, A_ub=np.concatenate((-coeff,coeff),axis=0), b_ub=np.concatenate((-betaF,k*betaF),axis=0),method='simplex')
C:\Users\karl\AppData\Local\Temp\ipykernel_16360\2694851566.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  REPAS=REPAS.append([REPAS.sum()])
C:\Users\karl\AppData\Local\Temp\ipykernel_16360\980770844.py:2: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  RESULTFb=so.linprog(z_ori, A_ub=np.concatenate((-coeff,coeff),axis=0), b_ub=np.concatenate((-betaF,k*betaF),axis=0),method='simplex')
C:\Users\karl\AppData\Local\Temp\ipyk


Pour les femmes avec des contraintes supplémentaires entre le minimum et 30 fois le minimum.

Pour les femmes avec des contraintes supplémentaires entre le minimum et 20 fois le minimum.

Pour les femmes avec des contraintes supplémentaires entre le minimum et 10 fois le minimum.

Pour les femmes avec des contraintes supplémentaires entre le minimum et 5 fois le minimum.

Repas impossible pour 4 fois le minimum

Repas impossible pour 3 fois le minimum

Repas impossible pour 2 fois le minimum

Repas impossible pour 1.5 fois le minimum

Repas impossible pour 1.4 fois le minimum

Repas impossible pour 1.3 fois le minimum

Repas impossible pour 1.2 fois le minimum

Repas impossible pour 1.1 fois le minimum

Repas impossible pour 1.09 fois le minimum

Repas impossible pour 1.08 fois le minimum

Repas impossible pour 1.07 fois le minimum


In [27]:
n=10
for ks in range(n):
    k=1.078611+0.000001*ks/n
    RESULTFb=so.linprog(z_ori, A_ub=np.concatenate((-coeff,coeff),axis=0), b_ub=np.concatenate((-betaF,k*betaF),axis=0),method='simplex') 
    if RESULTFb.success==True:
        print('\nPour les femmes avec des contraintes supplémentaires entre le minimum et {} fois le minimum.'.format(k))
        repas(RESULTFb)
    else : print('\nRepas impossible pour {} fois le minimum'.format(k))    


Repas impossible pour 1.078611 fois le minimum

Repas impossible pour 1.0786111 fois le minimum

Repas impossible pour 1.0786111999999999 fois le minimum

Repas impossible pour 1.0786113 fois le minimum

Repas impossible pour 1.0786114 fois le minimum

Repas impossible pour 1.0786115 fois le minimum

Repas impossible pour 1.0786116 fois le minimum

Repas impossible pour 1.0786117 fois le minimum

Repas impossible pour 1.0786118 fois le minimum

Repas impossible pour 1.0786119 fois le minimum


C:\Users\karl\AppData\Local\Temp\ipykernel_16360\90071053.py:4: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  RESULTFb=so.linprog(z_ori, A_ub=np.concatenate((-coeff,coeff),axis=0), b_ub=np.concatenate((-betaF,k*betaF),axis=0),method='simplex')
